## IR Assignment Group-082

## Group members

<style>
table {
  font-family: arial, sans-serif;
  border-collapse: collapse;
  width: 100%;
}

td, th {
  border: 1px solid #dddddd;
  text-align: left;
  padding: 8px;
}

tr:nth-child(even) {
  background-color: #dddddd;
}
</style>

<table>
  <tr>
    <th>Full Name</th>
    <th>BITS ID</th>
  </tr>
  <tr>
    <td>T V Sriharini</td>
    <td>2020fc04020@wilp.bits-pilani.ac.in</td>
  </tr>
  <tr>
    <td>Subodh Kant Mishra</td>
    <td>2020fc04064@wilp.bits-pilani.ac.in</td>
  </tr>
  <tr>
    <td>Mriganka Shekhar Gayen</td>
    <td>2020fc04069@wilp.bits-pilani.ac.in</td>
  </tr>
</table>

Given a Dataset collected from the Centre for Inventions and Scientific Information ("CISI") and consists of text data about 1,460 documents and 112 associated queries. Its purpose is to be used to build models of information retrieval where a given query will return a list of document IDs relevant to the query. The file "CISI.REL" contains the correct list (ie. "gold standard" or "ground proof") of query-document matching and your model can be compared against this "gold standard" to see how it has performed.

### 1. Load the data set and import necessary libraries (1 Marks)
We import various packages used for text and general data processing
* Pandas, Numpy, Skleaarn - For Data Processing
* NLTK - Natural Language Tool Kit for Natural Language Processing
* OS, functools - General Packages for Processing files


In [43]:
## Library imports
import numpy as np 
import pandas as pd
from rank_bm25 import BM25Okapi

from sklearn.feature_extraction.text import CountVectorizer

import os, glob, re, sys, random, unicodedata, collections
from tqdm import tqdm
from functools import reduce
import nltk
from collections import Counter

from nltk.corpus import stopwords
from nltk.stem import RSLPStemmer
from nltk.tokenize import sent_tokenize , word_tokenize
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

### 1.1 Importing the files - All FIle

We are importing the CISI.ALL file where we perform text processing such as removing newline characters, and '.' characters,
and split the data into different variables.

In [44]:
### Processing DOCUMENTS
doc_set = {}
doc_id = ""
doc_text = ""
with open('C://Users//HP//Downloads//CISI.ALL//CISI.ALL') as f:
    lines = ""
    for l in f.readlines():
        lines += "\n" + l.strip() if l.startswith(".") else " " + l.strip()
    lines = lines.lstrip("\n").split("\n")
doc_count = 0
for l in lines:
    if l.startswith(".I"):
        doc_id = int(l.split(" ")[1].strip())-1
    elif l.startswith(".X"):
        doc_set[doc_id] = doc_text.lstrip(" ")
        doc_id = ""
        doc_text = ""
    else:
        doc_text += l.strip()[3:] + " " # The first 3 characters of a line can be ignored. 

### 1.2 Importing the files - Queries FIle

We are importing the CISI.QRY file where we perform text processing such as removing newline characters, and '.' characters,
and split the data into different variables.

In [45]:
### Processing QUERIES
with open('C://Users//HP//Downloads//CISI.QRY') as f:
    lines = ""
    for l in f.readlines():
        lines += "\n" + l.strip() if l.startswith(".") else " " + l.strip()
    lines = lines.lstrip("\n").split("\n")
    
qry_set = {}
qry_id = ""
for l in lines:
    if l.startswith(".I"):
        qry_id = int(l.split(" ")[1].strip()) -1
    elif l.startswith(".W"):
        qry_set[qry_id] = l.strip()[3:]
        qry_id = ""

### 1.3 Importing the files - Rel FIle

We are importing the CISI.QREL file where we perform text processing such as removing newline characters, and '.' characters,
and split the data into different variables.

In [46]:
### Processing QRELS
rel_set = {}
with open('C://Users//HP//Downloads//CISI.REL') as f:
    for l in f.readlines():
        qry_id = int(l.lstrip(" ").strip("\n").split("\t")[0].split(" ")[0]) -1
        doc_id = int(l.lstrip(" ").strip("\n").split("\t")[0].split(" ")[-1])-1
        if qry_id in rel_set:
            rel_set[qry_id].append(doc_id)
        else:
            rel_set[qry_id] = []
            rel_set[qry_id].append(doc_id)

### 2.1 Extract features from text files. Each unique word in the document can be considered as a feature (1 Mark)

We check the General statistics of the file provided

In [47]:
## Here we check some statistics and info of CISI dataset

print('Read %s documents, %s queries and %s mappings from CISI dataset' % 
      (len(doc_set), len(qry_set), len(rel_set)))

number_of_rel_docs = [len(value) for key, value in rel_set.items()]
print('Average %.2f and %d min number of relevant documents by query ' % 
      (np.mean(number_of_rel_docs), np.min(number_of_rel_docs)))

print('Queries without relevant documents: ', 
      np.setdiff1d(list(qry_set.keys()),list(rel_set.keys())))

Read 1460 documents, 112 queries and 76 mappings from CISI dataset
Average 40.97 and 1 min number of relevant documents by query 
Queries without relevant documents:  [ 35  37  39  46  47  50  52  58  59  62  63  67  69  71  72  73  74  76
  77  79  82  84  85  86  87  88  90  92  93 102 104 105 106 107 109 111]


In [48]:
random.seed(42)
idx = random.sample(rel_set.keys(),1)[0]

print('Query ID %s ==>' % idx, qry_set[idx])
rel_docs = rel_set[idx]
print('Documents relevants to Query ID %s' % idx, rel_docs)
sample_document_idx = random.sample(rel_docs,1)[0]
print('Document ID %s ==>' % sample_document_idx, doc_set[sample_document_idx])

Query ID 14 ==> How much do information retrieval and dissemination systems, as well as automated libraries, cost? Are they worth it to the researcher and to industry?
Documents relevants to Query ID 14 [17, 26, 35, 48, 55, 58, 66, 73, 82, 125, 157, 163, 166, 191, 213, 221, 222, 249, 280, 291, 294, 298, 306, 330, 335, 337, 347, 364, 365, 366, 367, 371, 380, 445, 457, 464, 465, 481, 489, 490, 494, 496, 506, 519, 527, 590, 593, 622, 628, 638, 689, 719, 722, 723, 726, 727, 730, 778, 821, 833, 838, 847, 848, 864, 871, 896, 1099, 1160, 1247, 1304, 1352, 1357, 1362, 1365, 1367, 1370, 1371, 1373, 1374, 1375, 1376, 1409]
Document ID 48 ==> Adaptive Information Dissemination Sage, C.R. Anderson, R.R. Fitzwater, D.R. Computer dissemination of information offers significant advantages over manual dissemination because the computer can use strategies that are impractical and in some cases impossible for a human.. This paper describes the Ames Laboratory Selective Dissemination of Information syste

In [49]:
query = qry_set[idx] #get query text
rel_docs = rel_set[idx] #get relevant documents

# Index all documents using BM25
corpus = list(doc_set.values())
tokenized_corpus = [doc.split(" ") for doc in corpus]
bm25 = BM25Okapi(tokenized_corpus)

# Process query and get scores for each indexed document using BM25
tokenized_query = query.split(" ")
print('Query ==> ', query, '\nRelevant documents IDs: ==> ', rel_docs)
scores = bm25.get_scores(tokenized_query)
print(scores, len(scores), len(doc_set))

Query ==>  How much do information retrieval and dissemination systems, as well as automated libraries, cost? Are they worth it to the researcher and to industry? 
Relevant documents IDs: ==>  [17, 26, 35, 48, 55, 58, 66, 73, 82, 125, 157, 163, 166, 191, 213, 221, 222, 249, 280, 291, 294, 298, 306, 330, 335, 337, 347, 364, 365, 366, 367, 371, 380, 445, 457, 464, 465, 481, 489, 490, 494, 496, 506, 519, 527, 590, 593, 622, 628, 638, 689, 719, 722, 723, 726, 727, 730, 778, 821, 833, 838, 847, 848, 864, 871, 896, 1099, 1160, 1247, 1304, 1352, 1357, 1362, 1365, 1367, 1370, 1371, 1373, 1374, 1375, 1376, 1409]
[13.84492303 12.72656528 17.02184487 ... 12.72737224 14.26660278
 14.10298505] 1460 1460


In [50]:
##Source: https://gist.github.com/bwhite/3726239
def mean_reciprocal_rank(bool_results, k=10):
    """Score is reciprocal of the rank of the first relevant item
    First element is 'rank 1'.  Relevance is binary (nonzero is relevant).
    Example from http://en.wikipedia.org/wiki/Mean_reciprocal_rank
    >>> rs = [[0, 0, 1], [0, 1, 0], [1, 0, 0]]
    >>> mean_reciprocal_rank(rs)
    0.61111111111111105

    Args:
        rs: Iterator of relevance scores (list or numpy) in rank order
            (first element is the first item)
    Returns:
        Mean reciprocal rank
    """
    bool_results = (np.atleast_1d(r[:k]).nonzero()[0] for r in bool_results)
    return np.mean([1. / (r[0] + 1) if r.size else 0. for r in bool_results])

mean_reciprocal_rank([[0, 0, 1], [0, 1, 0], [1, 0, 0]])

0.611111111111111

In [51]:
## Argsort gives the indexes of values in increasing order, so we input with the negative values of scores
most_relevant_documents = np.argsort(-scores)

print(most_relevant_documents[:20]) # printing first 20 most relevant results

## Mask relevant documents with 0's and 1's according to query <-> document annotation
masked_relevance_results = np.zeros(most_relevant_documents.shape)
masked_relevance_results[rel_docs] = 1
sorted_masked_relevance_results = np.take(masked_relevance_results, most_relevant_documents)

print(sorted_masked_relevance_results[:20]) #printing first 20 results: 1 is relevant 0 isn't

# Calculate MRR@10
print(mean_reciprocal_rank([sorted_masked_relevance_results]))

[ 655  593  254  663  767  176  292  514  363  139   23  313 1026  460
 1447 1236  825  326 1276   27]
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
0.5


In [52]:
def results_from_query(qry_id, bm25):
    """Return an ordered array of relevant documents returned by query_id

    Args:
        qry_id (int): id of query on dataset
        bm25 (object): indexed corpus

    Returns:
        boolean sorted relevance array of documents
    """    
    query = qry_set[qry_id]
    query = qry_set[qry_id]
    rel_docs = []
    if qry_id in rel_set:
        rel_docs = rel_set[qry_id]
    tokenized_query = query.split(" ")
    scores = bm25.get_scores(tokenized_query)
    most_relevant_documents = np.argsort(-scores)
    masked_relevance_results = np.zeros(most_relevant_documents.shape)
  
    masked_relevance_results[rel_docs] = 1
    sorted_masked_relevance_results = np.take(masked_relevance_results, most_relevant_documents)
    
    return sorted_masked_relevance_results


results = [results_from_query(qry_id, bm25) for qry_id in list(qry_set.keys())]
print('MRR@10 %.4f' % mean_reciprocal_rank(results))

MRR@10 0.3146


In [53]:
# Instaciate objects from NLTK
stemmer = nltk.stem.PorterStemmer()
stop_words = nltk.corpus.stopwords.words('english')

In [54]:
def preprocess_string(txt, remove_stop=True, do_stem=True, to_lower=True):
    """
    Return a preprocessed tokenized text.
    
    Args:
        txt (str): original text to process
        remove_stop (boolean): to remove or not stop words (common words)
        do_stem (boolean): to do or not stemming (suffixes and prefixes removal)
        to_lower (boolean): remove or not capital letters.
        
    Returns:
        Return a preprocessed tokenized text.
    """    
    if to_lower:
        txt = txt.lower()
    tokens = nltk.tokenize.word_tokenize(txt)
    
    if remove_stop:
        tokens = [tk for tk in tokens if tk not in stop_words]
    if do_stem:
        tokens = [stemmer.stem(tk) for tk in tokens]
    return tokens

In [55]:
corpus = list(doc_set.values())
# You may experiment with this trying to improve MRR@10
remove_stop = True
do_stem = True
to_lower = True

tokenized_corpus = [preprocess_string(doc, remove_stop, do_stem, to_lower) for doc in corpus]

bm25 = BM25Okapi(tokenized_corpus)

def results_from_query_new(qry_id, bm25):
    query = qry_set[qry_id]
    rel_docs = []
    if qry_id in rel_set:
        rel_docs = rel_set[qry_id]
    tokenized_query = preprocess_string(query, remove_stop, do_stem, to_lower)
    scores = bm25.get_scores(tokenized_query)
    most_relevant_documents = np.argsort(-scores)
    masked_relevance_results = np.zeros(most_relevant_documents.shape)
corpus = list(doc_set.values())
# You may experiment with this trying to improve MRR@10
remove_stop = True
do_stem = True
to_lower = True

tokenized_corpus = [preprocess_string(doc, remove_stop, do_stem, to_lower) for doc in corpus]

bm25 = BM25Okapi(tokenized_corpus)

def results_from_query_new(qry_id, bm25):
    query = qry_set[qry_id]
    rel_docs = []
    if qry_id in rel_set:
        rel_docs = rel_set[qry_id]
    tokenized_query = preprocess_string(query, remove_stop, do_stem, to_lower)
    scores = bm25.get_scores(tokenized_query)
    most_relevant_documents = np.argsort(-scores)
    masked_relevance_results = np.zeros(most_relevant_documents.shape)
    masked_relevance_results[rel_docs] = 1
    sorted_masked_relevance_results = np.take(masked_relevance_results, most_relevant_documents)
    return sorted_masked_relevance_results


results = [results_from_query_new(qry_id, bm25) for qry_id in list(qry_set.keys())]
print('MRR@10 %.4f' % mean_reciprocal_rank(results))

MRR@10 0.4187
